In [3]:
import pandas as pd
import numpy as np

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [4]:
l = np.arange(1, 11)

out_file = "data/data_int.txt"
with open(out_file, 'w') as outfile:
    outfile.write(str(l) + '\n')

!cat data/data_int.txt

[ 1  2  3  4  5  6  7  8  9 10]


In [5]:
m = np.linspace(1, 25, 25, 'float')
m = m.reshape(5, 5)


out_file = "data/data_float.txt"
with open(out_file, 'w') as outfile:
    for i in range(len(m[:, 0])):
        for j in range(len(m[0, :])):
            outfile.write(str(m[i, j]))
            if j < len(m[0, :])-1:
                outfile.write(", ")
        outfile.write("\n")

!cat data/data_float.txt

1.0, 2.0, 3.0, 4.0, 5.0
6.0, 7.0, 8.0, 9.0, 10.0
11.0, 12.0, 13.0, 14.0, 15.0
16.0, 17.0, 18.0, 19.0, 20.0
21.0, 22.0, 23.0, 24.0, 25.0


In [6]:
import csv

with open("data/output.csv", "w") as f:
    writer = csv.writer(f)
    with open("data/data_float.txt", "r") as data_file:
        for line in csv.reader(data_file, delimiter = ','):
            writer.writerow(line)

2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [16]:
#!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json -P data/

In [17]:
import json
data = json.load(open("data/user_data.json"))

data_filtered = []
for i in range(len(data)):
    if data[i]["CreditCardType"] == "American Express":
        data_filtered.append(data[i])

headers = ["ID", "JobTitle", "EmailAddress", "FirstNameLastName", "CreditCard", "CreditCardType"] # set a headers string
line = [] # initialize line to write
with open("data/user_data_filtered.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(headers) # write the headers first
    for i in range(len(data_filtered)):
        for el in headers:
            line.append(data_filtered[i][el])
        writer.writerow(line) # write row (i) from data_filtered
        line = [] # clear variable line for reading row (i+1) from data_filtered


3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [20]:
#data = pd.read_csv("https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv")


4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [15]:
#!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat -P data/

In [ ]:
with open('data/credit_card.dat','rb') as file:
    file_content = file.read()
    word_counter = 0
    word_size = 8 # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter += 1
        if word_counter > 10: break
        word = struct.unpack('<q', file_content[i : i + word_size])[0] # get an 8-byte word
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        if i == 0: print ('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format('HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS'))
        print('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format(head, fpga, tdc_chan, orb_cnt, bx, tdc_meas))

5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3